In [181]:
# Image : ./downloadsIMG

In [182]:
import numpy as np

from PIL import Image

# Flask
from flask import Flask, request, render_template

# TensorFlow and tf.keras
from tensorflow.keras.applications.imagenet_utils import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image


In [183]:
# flask app
app = Flask(__name__)

# Modèle ImageNet
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

model = MobileNetV2(weights='imagenet')

In [184]:
def model_predict(img, model):
    img = img.resize((224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x, mode='tf')
    return model.predict(x)

In [185]:
def predict(img):
    preds = model_predict(img, model)
    pred_proba = "{:.3f}".format(np.amax(preds))
    pred_class = decode_predictions(preds, top=1)
    result = str(pred_class[0][0][1])
    result = result.replace('_', '').capitalize()
    return result
    

In [186]:
@app.route('/', methods=['POST', 'GET'])
def index():
    # Main page
    return render_template('index.html')

@app.route('/upload_image', methods=['GET', 'POST'])
def upload_image():
    # Vérifie si un fichier a été envoyé
    if 'image' not in request.files:
        return "No file part", 400
    file = request.files['image']
    
    # Vérifie si le fichier est valide
    if file.filename == '':
        return "No selected file", 400
    
    if file.content_type not in ['image/jpeg', 'image/png', 'image/gif']:
        return "Le fichier doit être une image au format JPEG, PNG ou GIF", 400
    
    try:
        # Ouvrir l'image en utilisant PIL
        image = Image.open(file.stream)
        print(type(image))
        # Si on arrive ici, l'image a bien été ouverte
        image.verify()  # Vérifie l'intégrité du fichier*
        
        prediction = predict(image)
        return f'{prediction}'

    except Image.UnidentifiedImageError:
        return "Erreur : le fichier n'est pas une image valide", 400

    except Exception as e:
        return f"Erreur lors de la manipulation de l'image : {e}", 500    

if __name__ == '__main__':
    app.run(host='localhost' ,port=5002, threaded=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://localhost:5002
Press CTRL+C to quit
127.0.0.1 - - [14/Nov/2024 15:08:58] "GET / HTTP/1.1" 200 -


<class 'PIL.WebPImagePlugin.WebPImageFile'>
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 804ms/step


127.0.0.1 - - [14/Nov/2024 15:09:03] "POST /upload_image HTTP/1.1" 200 -


0.958


127.0.0.1 - - [14/Nov/2024 15:09:06] "GET / HTTP/1.1" 200 -
